---

# 8. Generalized Parsing
**[Emil Sekerinski](http://www.cas.mcmaster.ca/~emil/), McMaster University, March 2021**

---

#### General Context-free Parsing

[Earley's parser](https://doi-org.libaccess.lib.mcmaster.ca/10.1145/362007.362035) works with an arbitrary context-free grammar without backtracking. If the grammar is unambiguous, it produces a parse tree in quadratic time; if the grammar is ambiguous, it produces all parse trees in cubic time (in the length of the input). For most “practical” grammars, it produces a parse tree in linear time.

We assume that the start symbol `S` appears only on the left-hand side of one rule, `S → π`; if that is not the case, a rule `S' → S` with a new start symbol `S'` can be added. Earley's parser is a top-down parser that constructs all possible derivations simultaneously: starting with `S`, nonterminals are eagerly expanded according to the all possible productions, rather than just a single production.

Let `P` be the set of productions and let the input be given by `x₁, …, xₙ`. Assume `xₙ₊₁ = $`, where `$` is a symbol that does not occur anywhere in the grammar. For each position of the input a set `sᵢ` of *Earley items* is maintained. An (Earley) item is a grammar rule with the right-hand side split, visualized by `•`, together with an index into the input string. An item `(A → σ • ω, j)`at position `i` means that `A` is attempted to be recognized at input position `j + 1` and up to `i` the input `xⱼ₊₁…xᵢ` can be derived from `σ`, formally `σ ⇒* xⱼ₊₁…xᵢ`. At each position `i`, the algorithm adds items to `sᵢ` in *predict* and *complete* steps and to `sᵢ₊₁` in *match* steps. The algorithm iterates over all items at one position. Since items are being added, a set, `v`, of visited items is maintained.

```
s₀ := {(S → • π, 0)}; for i = 1 to n do sᵢ := {}
for i = 0 to n do
    v := {}
    while v ≠ sᵢ do
        e :∈ sᵢ - v; v := v ∪ {e}
        case e of
            (A → σ • a ω, j) and a = xᵢ₊₁:        -- match (M)
                sᵢ₊₁ := sᵢ₊₁ ∪ {(A → σ a • ω, j)} 
            (A → σ • B ω, j):                            -- predict (P)
                for B → μ ∈ P do
                    sᵢ := sᵢ ∪ (B → • μ, i) 
            (A → σ •, j):                                   -- complete (C)
                for (B → μ • A ξ, k) ∈ sⱼ do
                    sᵢ := sᵢ ∪ {(B → μ A • ξ, k)}
accept := (S → π •, 0) ∈ sₙ
```

<div style="float:left">

Consider the grammar:

    E → T | E + T
    T → F | T × F
    F → a

The input `a + a × a` is accepted as `(S → E •, 0) ∈ s₅`.

Lines in bold correspond to the derivation.
</div>

<div style="float:right">

|            | item                | step      |
|:-----------|:--------------------|:----------|
| `s₀`:      | `S → • E, 0`        |           |
| `(x₁ = a)` | `E → • T, 0`        | P         |
|            | `E → • E + T, 0`    | P         |
|            | `T → • F, 0`        | P         |
|            | `T → • T × F, 0`    | P         | 
|            | `F → • a, 0`        | P         |
| `s₁`:      | **`F → a •, 0`**        | **M at `0`**  |
| `(x₂ = +)` | **`T → F •, 0`**        | C         |
|            | **`E → T •, 0`**        | C         |
|            | `T → T • × F, 0`    | C         |
|            | `S → E •, 0`        | C         |
|            | `E → E • + T, 0`    | C         |
| `s₂`:      | `E → E + • T, 0`    | M at `1`  |
| `(x₃ = a)` | `T → • T × F, 2`    | P         |
|            | `T → • F, 2`        | P         |
|            | `F → • a, 2`        | P         |
| `s₃`:      | **`F → a •, 2`**        | **M at `2`**  |
| `(x₄ = ×)` | **`T → F •, 2`**        | **C**         |
|            | `E → E + T •, 0`    | C         |
|            | `T → T • × F, 2`    | C         |
|            | `S → E •, 0`        | C         |
|            | `E → E • + T, 0`    | C         |
| `s₄`:      | `T → T × • F, 2`    | M at `3`  |
| `(x₅ = a)` | `F → • a, 4`        | P         |
| `s₅`:      | **`F → a •, 4`**        | **M at `4`**  |
| `(x₆ = $)` | **`T → T × F •, 2`**    | **C**         |
|            | **`E → E + T •, 0`**    | C         |
|            | `T → T • × F, 2`    | C         |
|            | **`S → E •, 0`**        | **C**         |
|            | `E → E • + T, 0`    | C         |

</div>

The Python implementation below assumes that each terminal and nonterminal is a single character, the grammar is represented by a tuple of productions, and each production is a string of the form `A→τ` where `A` is a nonterminal. The first production, `g[0]` in the implementation, defines the start symbol. Since in Python strings are indexed starting from `0`, an extra character, `^`, is prepended to the input. The sequence `a ω` in the algorithm corresponds to `τ` in the implementation and `A ξ` corresponds to `ν`. 

In [1]:
def parse(g: "grammar", x: "input"):
    global s
    n = len(x); x = '^' + x + '$'; S, π = g[0][0], g[0][2:]
    s = [{(S, '', π, 0)}] + [set() for _ in range(n)]#; print('   s[ 0 ]:', S, '→ •', π, ', 0')
    for i in range(n + 1):
        v = set() # visited items
        while v != s[i]:
            e = (s[i] - v).pop(); v.add(e) # pick an arbirary un-visited item
            A, σ, τ, j = e
            if len(τ) > 0 and τ[0] == x[i + 1]: # match, a == τ[0]
                f = (A, σ + τ[0], τ[1:], j)
                s[i + 1].add(f)#; print('M  s[', i + 1, ']:', f[0], '→', f[1], '•', f[2], ',', f[3])
            elif len(τ) > 0: # predict, B == ω[0]
                for f in ((r[0], '', r[2:], i) for r in g if r[0] == τ[0]):
                    s[i].add(f)#; print('P  s[', i, ']:', f[0], '→', f[1], '•', f[2], ',', f[3])
            else: # complete, len(τ) == 0
                for f in ((B, μ + ν[0], ν[1:], k) for (B, μ, ν, k) in s[j] if len(ν) > 0 and ν[0] == A):
                    s[i].add(f); #print('C  s[', i, ']:', f[0], '→', f[1], '•', f[2], ',', f[3])
    return (S, π, '', 0) in s[n]

In [2]:
G1 = ("S→E", "E→a", "E→E+E")

In [3]:
parse(G1, "a+a+a")

True

In [ ]:
grammar = ("S→E", "E→T", "E→E+T", "T→F", "T→T×F", "F→a")

In [ ]:
parse(grammar, "a+a×a")

The algorithm can be "animated" by uncommenting the `print` statements; the resulting set of items can also be observed:

In [4]:
s

[{('E', '', 'E+E', 0), ('E', '', 'a', 0), ('S', '', 'E', 0)},
 {('E', 'E', '+E', 0), ('E', 'a', '', 0), ('S', 'E', '', 0)},
 {('E', '', 'E+E', 2), ('E', '', 'a', 2), ('E', 'E+', 'E', 0)},
 {('E', 'E', '+E', 0),
  ('E', 'E', '+E', 2),
  ('E', 'E+E', '', 0),
  ('E', 'a', '', 2),
  ('S', 'E', '', 0)},
 {('E', '', 'E+E', 4),
  ('E', '', 'a', 4),
  ('E', 'E+', 'E', 0),
  ('E', 'E+', 'E', 2)},
 {('E', 'E', '+E', 0),
  ('E', 'E', '+E', 2),
  ('E', 'E', '+E', 4),
  ('E', 'E+E', '', 0),
  ('E', 'E+E', '', 2),
  ('E', 'a', '', 4),
  ('S', 'E', '', 0)}]

For efficiency, instead of using a set of items for an Earley state, a lists with a marker separating the items  that have been visited and that still need to be visited can be used.

The number of items in `sᵢ` is proportional to `i` in the worst case. Matching and predicting need at most `i` steps for `sᵢ`, but completing may need `i²` steps, as adding an item may cause a previous set to be revisited. Summing `i²` for `i` from `0` to `n` is `n³`, thus Earley's parser needs `n³` steps in the worst case.